In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 588.3 MB 18 kB/s 
     |████████████████████████████████| 1.7 MB 72.6 MB/s 
     |████████████████████████████████| 439 kB 61.3 MB/s 
     |████████████████████████████████| 6.0 MB 48.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')

df = df[df.Rating != 3]

df['is_positive'] = (df['Rating'] >= 4).astype(int)

In [ ]:
df.drop(columns=['Rating'], inplace=True)

In [ ]:
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df

,DATA_COLUMN,LABEL_COLUMN
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1
5,love monaco staff husband stayed hotel crazy w...,1
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",1
20487,great location price view hotel great quick pl...,1
20488,"ok just looks nice modern outside, desk staff ...",0
20489,hotel theft ruined vacation hotel opened sept ...,0


In [ ]:
df['LABEL_COLUMN'].value_counts()

1    15093
0     3214
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]

In [ ]:
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
# Тестовая выборка

In [ ]:
n_test = (df_negative.shape[0] // 4) * 3 # в оригинале (df_negative.shape[0] // 4) * 3
df_negative_test = df_negative.tail(n_test)
n_test = (df_positive.shape[0] // 20) * 3 # в оригинале df_positive.shape[0] // 20) * 3
df_positive_test = df_positive.tail(n_test)

In [ ]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
17565,wonderful hotel wonderful town unbeknownst man...,1
17566,wonderful wonderful wonderful just returned ye...,1
17567,"affinia 50 you'all, stayed affinia 50 week, li...",1
17568,just right good location walking distance bloo...,1
17569,loved place want thank outstanding service goi...,1
...,...,...
20480,great play stay stay loyal inn package deal ha...,1
20482,great choice wife chose best western quite bit...,1
20483,good bed clean convenient just night happy sta...,1
20486,"best kept secret 3rd time staying charm, not 5...",1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test.sample(10)

,DATA_COLUMN,LABEL_COLUMN
19467,"loved palace stayed palace feb. 11 18th, time ...",1
16406,really really arrived aston ubud waved securit...,0
18514,loved majestic colonial punta cana inclusive h...,1
18669,"perfect, truly leading hotel luxury relatively...",1
19153,n't ask better based tripadvisor rating booked...,1
18330,wonderful stay moment walked reception staff h...,1
6154,"not clean just came 7-day stay sky court, loca...",0
18174,"fabulous, sister just returned following 3 day...",1
19244,not stay dallas victory w stayed w treat lovel...,0
18865,nice convenient hong kong disney suggest stay ...,1


In [ ]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2409
1    2262
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Обучающая выборка

In [ ]:
n_train = df_negative.shape[0] // 4 # в оригианле df_negative.shape[0] // 4
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 20 # в оригинале df_positive.shape[0] // 20
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
df_balanced_train['LABEL_COLUMN'].value_counts()

0    803
1    754
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_balanced_train.sample(10)

,DATA_COLUMN,LABEL_COLUMN
513,"smells like sewer, summary tiny rooms toilet/s...",0
2725,star ghetto motel hi arrived ant recomended su...,0
3906,"ponce hilton- not cracked, february husband de...",0
348,unclean terrible service stayed save money 3 d...,0
2038,disappointing husband stayed doubltree april m...,0
560,"going cuz loved, originally booked hotel valen...",1
792,n't fooled pictures 12 hour road trip friends ...,0
414,"great stay w seattle, stayed july boyfriend 20...",1
619,"simple perfect stayed times disappointed, room...",1
197,"agree worst hilton, absolute worst hilton stay...",0


In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [ ]:
distilbert_preprocess = hub.KerasLayer('https://tfhub.dev/jeongukjae/distilbert_en_uncased_preprocess/2')

In [ ]:
distilbert_encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_en_uncased_L-6_H-768_A-12/1", trainable=False)

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = distilbert_preprocess(text_input)
outputs = distilbert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                      
                                                                                                  
 keras_layer_1 (KerasLayer)     {'pooled_output': (  66362880    ['keras_layer[0][0]',        

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
49/49 [==============================] - 349s 7s/step - loss: 0.6345 - accuracy: 0.6795 - precision: 0.6576 - recall: 0.7056
Epoch 2/10
49/49 [==============================] - 343s 7s/step - loss: 0.4819 - accuracy: 0.8542 - precision: 0.8685 - recall: 0.8236
Epoch 3/10
49/49 [==============================] - 344s 7s/step - loss: 0.4045 - accuracy: 0.8780 - precision: 0.8801 - recall: 0.8660
Epoch 4/10
49/49 [==============================] - 345s 7s/step - loss: 0.3608 - accuracy: 0.8786 - precision: 0.8772 - recall: 0.8714
Epoch 5/10
49/49 [==============================] - 340s 7s/step - loss: 0.3277 - accuracy: 0.8960 - precision: 0.8936 - recall: 0.8912
Epoch 6/10
49/49 [==============================] - 342s 7s/step - loss: 0.3080 - accuracy: 0.8979 - precision: 0.9059 - recall: 0.8806
Epoch 7/10
49/49 [==============================] - 342s 7s/step - loss: 0.2918 - accuracy: 0.8998 - precision: 0.9019 - recall: 0.8899
Epoch 8/10
49/49 [==============================

In [ ]:
y_predicted = model.predict(X_test)
y_predicted

146/146 [==============================] - 1025s 7s/step


array([[0.11208095],
       [0.26394412],
       [0.11142197],
       ...,
       [0.5208257 ],
       [0.8633075 ],
       [0.9907745 ]], dtype=float32)

In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
y_test

4133     0
4134     0
4135     0
4136     0
4140     0
        ..
20480    1
20482    1
20483    1
20486    1
20487    1
Name: LABEL_COLUMN, Length: 4671, dtype: int64

In [ ]:
accuracy_score(y_test, y_predicted)

0.873260543780775

In [ ]:
precision_score(y_test, y_predicted)


0.900287631831256

In [ ]:
recall_score(y_test, y_predicted)

0.830238726790451

In [ ]:
f1_score(y_test, y_predicted)

0.8638454461821528

In [ ]:
df_results_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test, y_predicted)


In [ ]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/distilbert_trained_on_hotel_reviews_25_december'

In [ ]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/distilbert_trained_on_hotel_reviews_25_december


In [ ]:
model.save(saved_model_path, include_optimizer=True) 

In [ ]:
df_results_for_reviews_on_hotels


,accuracy,precision,recall,f1_score
reviews on hotels,0.873261,0.900288,0.830239,0.863845
